In [ ]:
#Data Science
#Coder: Marco Galicia
#Alma mater: IPN Mexico - ESIME ICE
#Seminario: Patrones Area Médica 2019
#Work based on:
#Hands on ML - Aurelien Geron - O'reilly media Inc,
#https://www.kaggle.com/sarques/aiwitbor/notebook
#https://kapernikov.com/tutorial-image-classification-with-scikit-learn/
#https://towardsdatascience.com/new-to-cnn-learn-your-basics-here-with-the-mnist-digit-recognizer-dataset-538313a29c88
#https://becominghuman.ai/understanding-the-basics-of-cnn-with-image-classification-7f3a9ddea8f9
#"Yo solo sé que no se nada"...olvida todas las palabras griegas y los rollos de papiro...
import numpy as np # linear algebra
from numpy import array
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

print(os.listdir("../input/skin-cancer-mnist-ham10000"))
base_add = os.path.join('..', 'input/skin-cancer-mnist-ham10000')
print(base_add)
# Any results you write to the current directory are saved as output.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
from glob import glob
%matplotlib inline
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import itertools
import keras
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, Activation
from keras.layers import MaxPooling2D, BatchNormalization
import tensorflow as tf

In [ ]:
#modulo 3
#https://stackoverflow.com/questions/8532929/two-asterisks-in-file-path
image_path_dict = {os.path.splitext(os.path.basename(x))[0]: 
                   x for x in glob(os.path.join(base_add, '*', '*.jpg'))}
print(len(image_path_dict))
lesion_type_dict = {'nv': 'Melanocytic nevi',
    'mel': 'Melanoma',
    'bkl': 'Benign keratosis-like lesions ',
    'bcc': 'Basal cell carcinoma',
    'akiec': 'Actinic keratoses',
    'vasc': 'Vascular lesions',
    'df': 'Dermatofibroma'}
df = pd.read_csv(os.path.join(base_add, 'HAM10000_metadata.csv'))
print("tipo de objeto:",type(df))
print("longitud:",len(df))

In [ ]:
#modulo 4
#operacion en bruto, se observa el comportamiento de series.map...
#si en df['dx'].iloc[0] = 'bkl', map busca y encuentra 'Benign keratosis-like lesions'
#si en df['dx'].iloc[10010] = 'akeic', map busca y encuentra 'Actinic keratoses'
#busqueda y asignacion com map key:value
df['cell_type'] = df['dx'].map(lesion_type_dict.get)
df['path'] = df['image_id'].map(image_path_dict.get)
#series(columnas) del data frame despues de la asignacion
print(df.columns)
df.head(3)

In [ ]:
#modulo 5
#analisis datos para preparar entrenamiento supervisado
#crea categorias para (dx): 7 to many, nueva serie/columna dx_code
df['dx_code'] = pd.Categorical(df['dx']).codes
print(pd.Categorical(df['dx']).unique())
print(pd.Categorical(df['dx_code']).unique())
#confirmando...
#bkl=2; nv=5; df=3; mel=4; vasc=6; bcc=1; akeic=0;
#df.sample(7)
df.head(10)

In [ ]:
#homogeneizar
df['age'].fillna(df['age'].mean(), inplace = True)
df.info()

In [ ]:
#https://stackoverflow.com/questions/43027980/purpose-of-matplotlib-inline
df['cell_type'].value_counts().plot(kind = 'bar')
print(df['cell_type'].value_counts())

In [ ]:
#modulo 8
df['image_3d'] = df['path'].map(lambda x: np.asarray(Image.open(x).resize((100, 75))))
print(df.columns)
print(df['image_3d'].shape)
df.head(3)

In [ ]:
print('Numero de muestras: ', len(df['image_3d']))
print('keys(columnas): ', list(df.keys()))
print('Descripcion: ', df['cell_type'])
print('image shape: ', df['image_3d'][0].shape)
print('Etiquetas num(labels):', np.unique(df['dx_code']))

In [ ]:
#Confirmando la categorizacion por tipo de lesion
# valores unico en la serie dx_code (etiquetas-label)
labels = np.unique(df['dx_code'])
 
# set up the matplotlib figure and axes, based on the number of labels RGB & GRAYscale
fig, axes = plt.subplots(1, len(labels))
fig.set_size_inches(15,4)
fig.tight_layout()
          
#https://www.digitalocean.com/community/tutorials/how-to-use-break-continue-and-pass-statements-when-working-with-loops-in-python-3
for axRGB, dx_code in zip(axes, labels):
    #print(dx_code)
    for j in range(len(df['image_3d'])):
        if df['dx_code'].iloc[j] == dx_code:
            #print("found:",df['dx_code'].iloc[j])
            axRGB.imshow(df['image_3d'].iloc[j])
            axRGB.axis('off')
            axRGB.set_title(dx_code)
            break

In [ ]:
X = np.array(df['image_3d'])
y = np.array(df['dx_code'])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    shuffle=True,
    random_state=42,
)

In [ ]:
#hyperparametros CNN
batch_size = 32
num_classes = 7
epochs = 5
input_shape = (75, 100, 3)

In [ ]:
#mODULO 14: Convertir lista(vector) a array(matrix) binario(patron binario).
print("Vector->Lista:",y_test)
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print("Matrix -> Array Binario->:",y_test)

In [ ]:
#Normalization max y min
X_train = np.asarray(X_train.tolist())
X_test = np.asarray(X_test.tolist())

print("Antes de Normalizar")
print("maixmos:",np.amin(X_train))
print("minimos:",np.amax(X_train))

X_train_mean = np.mean(X_train)
X_test_mean = np.mean(X_test)

X_train_std = np.std(X_train)
X_test_std = np.std(X_test)

X_train = (X_train - X_train_mean)/X_train_std
X_test = (X_test - X_test_mean)/X_test_std

print("Despues de Normalizar")
print("maixmos:",np.amin(X_train))
print("minimos:",np.amax(X_train))

In [ ]:
# model architecture
model = Sequential()
model.add(Conv2D(32, (3, 3),padding='same',input_shape=input_shape))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [ ]:
# summary
model.summary()

In [ ]:
# compile the model
model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

In [ ]:
# ajuste del modelo
model.fit(X_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(X_test, y_test),
              shuffle=True)

#################
#y_pred=model.predict_classes(X_test)
#print(y_pred)

In [ ]:
#converir clases de fold validacion y_test: binario a decimal
Y_true = np.argmax(y_test,axis = 1) 
#predicciones del modelo CNN sobre X_test, estimado para y
y_pred_classes=model.predict_classes(X_test)
print("conteo de validacion")
print(pd.value_counts(Y_true))
#Computo de la matriz de confusion
confusion_mtx = confusion_matrix(Y_true, y_pred_classes)  
print("matrix de confusion")
print(confusion_mtx)

In [ ]:
plt.matshow(confusion_mtx, cmap=plt.cm.gray)
plt.show()

Inferencia CNN: La grafica es estatica, muestra la diagonal de la matriz de confusion vs los datos de validacion.

In [ ]:
#version estatica 5 epocas
import matplotlib
import matplotlib.pyplot as plt
import numpy as np


labels = ['akeic', 'bcc', 'bkl', 'df', 'mel', 'nv' , 'vasc']
validacion = [69, 93, 228, 28, 226 , 1338, 21 ]
inferencia = [16, 48, 96, 0, 17, 1287, 9]

x = np.arange(len(labels))  # the label locations
width = 0.35  # the width of the bars

fig, ax = plt.subplots()
rects1 = ax.bar(x - width/2, validacion, width, label='Conteo Validación')
rects2 = ax.bar(x + width/2, inferencia, width, label='Conteo inferencia')

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Conteo de lesiones')
ax.set_title('Conteo autómata CNN ≈ 73.54%')
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.legend()


def autolabel(rects):
    """Attach a text label above each bar in *rects*, displaying its height."""
    for rect in rects:
        height = rect.get_height()
        ax.annotate('{}'.format(height),
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3),  # 3 points vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom')


autolabel(rects1)
autolabel(rects2)

fig.tight_layout()

plt.show()